In [1]:
import statsapi as mlb
import pandas as pd
import sqlalchemy
import config
import csv
from os import walk
import re
from datetime import datetime
import functions
import sql_alch_schema
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,DateTime,Date,Time,Boolean,Float
from sqlalchemy import ForeignKey,and_
from sqlalchemy.orm import relationship


_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
## Create the engine 
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True,})

Base = sql_alch_schema.Base

Game = sql_alch_schema.Game
Play = sql_alch_schema.Play
Team = sql_alch_schema.Team
GameTeamLink = sql_alch_schema.GameTeamLink
Person = sql_alch_schema.Person



from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()


In [ ]:
#Base.metadata.create_all(db)

In [2]:
game_pk_dict=functions.read_gamePks()

In [14]:
# games_2018 = game_pk_dict['2018']
# games_2018 = [int(x) for x in games_2018]
# game_ids = [item for sublist in session.query(Game.pk).all() for item in sublist]
# games_to_add = [x for x in games_2018 if x not in game_ids]

In [17]:
# print(len(games_to_add))
# [sql_alch_schema.create_gameRecord_playsRecords(x,session) for x in games_to_add]

996


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [3]:
# current_team_ids = {item for sublist in session.query(Team.id).all() for item in sublist}
# new_team_ids = {item for sublist in session.query(Game.homeTeam_id,Game.awayTeam_id).all() for item in sublist}
# new_team_ids = new_team_ids-current_team_ids
# list(new_team_ids)

[]

In [19]:
# sql_alch_schema.create_addTeam(list(new_team_ids),session)

In [4]:
current_gameTeams = {item for sublist in session.query(GameTeamLink.game_id).all() for item in sublist}
current_games = {item for sublist in session.query(Game.id).all() for item in sublist}
gameTeams_to_add = current_games-current_gameTeams
gameTeams_to_add = list(gameTeams_to_add)
len(gameTeams_to_add)

0

In [30]:
# def get_roster_inputs(query):   
#     roster_inputs = []
#     for instance in query.all():
#         roster_input_dict = {'date':datetime.strftime(instance.dateTime,'%Y-%m-%d'),
#                              'season':instance.season,
#                              'homeTeam':instance.homeTeam_id,
#                              'awayTeam':instance.awayTeam_id,
#                              }
#         roster_inputs.append(roster_input_dict)
        
#     return roster_inputs

# def get_roster(roster_input_dict):
#     #player_list = ['player_'+str(x) for x in range(1,41)]
#     home = mlb.get('team_roster',
#                    {'teamId':roster_input_dict['homeTeam'],
#                     'rosterType':'active',
#                     'season':roster_input_dict['season'],
#                     'date':roster_input_dict['date']
#                    })['roster']
#     home_roster_ids = [x['person']['id'] for x in home]
#     player_list = ['player_'+str(x) for x in range(1,len(home_roster_ids)+1)]
#     home_roster_dict = {x:y for x,y in zip(player_list,home_roster_ids)}
#     home_roster_dict['teamId'] = roster_input_dict['homeTeam']
    
#     away = mlb.get('team_roster',
#                    {'teamId':roster_input_dict['awayTeam'],
#                     'rosterType':'active',
#                     'season':roster_input_dict['season'],
#                     'date':roster_input_dict['date']
#                    })['roster']
#     away_roster_ids = [x['person']['id'] for x in away]
#     player_list = ['player_'+str(x) for x in range(1,len(away_roster_ids)+1)]
#     away_roster_dict = {x:y for x,y in zip(player_list,away_roster_ids)}
#     away_roster_dict['teamId']=roster_input_dict['awayTeam']
#     return home_roster_dict,away_roster_dict

# def create_GameTeamLink(game_ids):
#     records = []
#     for game_id in game_ids:
#         game_query = session.query(Game).filter_by(id=game_id)
#         roster_input_dicts = get_roster_inputs(game_query)
#         home_roster,away_roster = get_roster(roster_input_dicts[0])
#         rosters = [home_roster,away_roster]
# #       rosters.append(roster)
        
#         for roster in rosters:
#             game_team_record = GameTeamLink(game_id=game_id,
#                                             team_id=roster['teamId'],

#                                             player_1_id = roster.get('player_1','null'),
#                                             player_2_id = roster.get('player_2','null'),
#                                             player_3_id = roster.get('player_3','null'),
#                                             player_4_id = roster.get('player_4','null'),
#                                             player_5_id = roster.get('player_5','null'),
#                                             player_6_id = roster.get('player_6','null'),
#                                             player_7_id = roster.get('player_7','null'),
#                                             player_8_id = roster.get('player_8','null'),
#                                             player_9_id = roster.get('player_9','null'),
#                                             player_10_id = roster.get('player_10','null'),
#                                             player_11_id = roster.get('player_11','null'),
#                                             player_12_id = roster.get('player_12','null'),
#                                             player_13_id = roster.get('player_13','null'),
#                                             player_14_id = roster.get('player_14','null'),
#                                             player_15_id = roster.get('player_15','null'),
#                                             player_16_id = roster.get('player_16','null'),
#                                             player_17_id = roster.get('player_17','null'),
#                                             player_18_id = roster.get('player_18','null'),
#                                             player_19_id = roster.get('player_19','null'),
#                                             player_20_id = roster.get('player_20','null'),
#                                             player_21_id = roster.get('player_21','null'),
#                                             player_22_id = roster.get('player_22','null'),
#                                             player_23_id = roster.get('player_23','null'),
#                                             player_24_id = roster.get('player_24','null'),
#                                             player_25_id = roster.get('player_25','null'),
#                                             player_26_id = roster.get('player_26','null'),
#                                             player_27_id = roster.get('player_27','null'),
#                                             player_28_id = roster.get('player_28','null'),
#                                             player_29_id = roster.get('player_29','null'),
#                                             player_30_id = roster.get('player_30','null'),
#                                             player_31_id = roster.get('player_31','null'),
#                                             player_32_id = roster.get('player_32','null'),
#                                             player_33_id = roster.get('player_33','null'),
#                                             player_34_id = roster.get('player_34','null'),
#                                             player_35_id = roster.get('player_35','null'),
#                                             player_36_id = roster.get('player_36','null'),
#                                             player_37_id = roster.get('player_37','null'),
#                                             player_38_id = roster.get('player_38','null'),
#                                             player_39_id = roster.get('player_39','null'),
#                                             player_40_id = roster.get('player_40','null')
#                                            )
#             records.append(game_team_record)
#     return records

# def chunk(n,list_to_chunk):
#     """
#     takes in n, and a list to chunk. returns a list of lists with n length. The last chunk size may or may not 
#     be equal to n. 
#     """
#     return [ list_to_chunk[i:i+n] for i in range(0,len(list_to_chunk),n) ]

# def create_add_GameTeamLink(session,start=0,stop=None,chunk_size=50):   
#     # collect game_ids from the games table 
#     ids_list_test = session.query(Game.id).all()[start:stop]
#     ids_list_test=[item for sublist in ids_list_test for item in sublist]

#     # collect game_ids from the game_link table
#     # I'll naturally have duplicates so I think I'll make it a set 
#     already_added = list({item for sublist in session.query(GameTeamLink.game_id).all() for item in sublist})

#     games_to_get = [game for game in ids_list_test if game not in already_added]
    
#     list_of_chunks = chunk(chunk_size,games_to_get)
#     count = 1
    
#     for _chunk in list_of_chunks:
#         try:
#             print(f'starting chunk {count} out of {len(list_of_chunks)}')
#             games_teams_to_add = create_GameTeamLink(_chunk)

#             session.add_all(games_teams_to_add)
#             session.commit()

#             count = count+1
#         except:
#             print('chunk failed. Rolling back the session and trying the next chunk')
#             session.rollback()
#             continue 
# create_add_GameTeamLink(session)

starting chunk 1 out of 6
starting chunk 2 out of 6
starting chunk 3 out of 6
starting chunk 4 out of 6
starting chunk 5 out of 6
starting chunk 6 out of 6


In [6]:
current_people = {item for sublist in session.query(Person.id).all() for item in sublist}
new_people = {item for sublist in session.query(Play.pitcher_id).all() for item in sublist}
new_people = list(new_people-current_people)
new_people

[]

In [39]:
def create_personRecord(personId):
    api_call = mlb.get('person',{'personId':personId})

    person = api_call['people'][0]
    person['birthDate'] = person.get('birthDate','1900-01-01')

    
    person_record = Person(id=person.get('id','null'),
                           fullName=person.get('fullName','null'),
                           firstName=person.get('firstName','null'),
                           lastName=person.get('lastName','null'),
                           primaryNumber=person.get('primaryNumber','null'),
                           birthDate=datetime.strptime(person.get('birthDate','1900-01-01'),'%Y-%m-%d'),
                           currentAge=person.get('currentAge','null'),
                           birthCity=person.get('birthCity','null'),
                           birthCountry=person.get('birthCountry','null'),
                           height=person.get('height','null'),
                           weight=person.get('weight','null'),
                           active=person.get('active','null'),
                           primaryPosition_code=person.get('primaryPosition',{'code':'null'})['code'],
                           primaryPosition_name=person.get('primaryPosition',{'name':'null'})['name'],
                           primaryPosition_type=person.get('primaryPosition',{'type':'null'})['type'],
                           primaryPosition_abbreviation=person.get('primaryPosition',{'abbreviation':'null'})['abbreviation'],
                           gender=person.get('gender','null'),
                           isPlayer=person.get('isPlayer','null'),
                           isVerified=person.get('isVerified','null'),
                           draftYear=person.get('draftYear','null'),
                           mlbDebutDate=datetime.strptime(person.get('mlbDebutDate','1900-01-01'),'%Y-%m-%d'),
                           batSide=person.get('batSide',{'description':'null'})['description'],
                           pitchHand=person.get('pitchHand',{'description':'null'})['description'],
                           nameSlug=person.get('nameSlug','null'),
                           fullFMLName=person.get('fullFMLName','null'),
                           strikeZoneTop=person.get('strikeZoneTop','null'),
                           strikeZoneBottom=person.get('strikeZoneBottom','null'),
                          )
    return person_record

def create_addPerson(session,personIds,chunk_size=50):
    already_added = [item for sublist in session.query(Person.id).all() for item in sublist]
    records=[]
    
    people_to_get = [person for person in personIds if person not in already_added]
    
    list_of_chunks = chunk(chunk_size,people_to_get)
    count=1
    
    for _chunk in list_of_chunks:
        try:
            print(f'starting chunk {count} out of {len(list_of_chunks)}')
            records = [create_personRecord(x) for x in _chunk]
            session.add_all(records)
            session.commit()
            count+=1
        except:
            print('chunk failed. rolling back session and trying the next chunk')
            session.rollback()
            count+=1
            continue
            


In [41]:
#create_addPerson(session,new_people)

In [4]:
mlb.ENDPOINTS['meta']

{'url': 'https://statsapi.mlb.com/api/{ver}/{type}',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'type': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': [[]],
 'required_params': [[]],
 'note': 'The meta endpoint is used to retrieve values to be used within other API calls. Available types: awards, baseballStats, eventTypes, gameStatus, gameTypes, hitTrajectories, jobTypes, languages, leagueLeaderTypes, logicalEvents, metrics, pitchCodes, pitchTypes, platforms, positions, reviewReasons, rosterTypes, scheduleEventTypes, situationCodes, sky, standingsTypes, statGroups, statTypes, windDirection.'}

In [ ]:
mlb.player_stats()

In [47]:
stats_test= mlb.get('stats',{'playerId':527054,'group':'hitting','stats':'career'})

In [49]:
stats_test['stats']

[{'type': {'displayName': 'career'},
  'group': {'displayName': 'hitting'},
  'totalSplits': 1017,
  'exemptions': [],
  'splits': [{'stat': {'gamesPlayed': 3035,
     'airOuts': 0,
     'runs': 2246,
     'doubles': 723,
     'triples': 297,
     'homeRuns': 117,
     'strikeOuts': 357,
     'baseOnBalls': 1249,
     'hits': 4191,
     'hitByPitch': 94,
     'avg': '.367',
     'atBats': 11429,
     'obp': '.433',
     'slg': '.513',
     'ops': '.946',
     'caughtStealing': 178,
     'stolenBases': 892,
     'stolenBasePercentage': '.834',
     'plateAppearances': 13067,
     'totalBases': 5859,
     'rbi': 1938,
     'sacBunts': 295,
     'babip': '.372',
     'atBatsPerHomeRun': '97.68'},
    'team': {'id': 133,
     'name': 'Philadelphia Athletics',
     'link': '/api/v1/teams/133'},
    'player': {'id': 112431,
     'fullName': 'Ty Cobb',
     'link': '/api/v1/people/112431',
     'lastName': 'Cobb'},
    'league': {'id': 103, 'name': 'AL', 'link': '/api/v1/league/103'},
    'sp

In [4]:
mlb.ENDPOINTS['person_stats']

{'url': 'https://statsapi.mlb.com/api/{ver}/people/{personId}/stats/game/{gamePk}',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'personId': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'gamePk': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': ['fields'],
 'required_params': [[]],
 'note': 'Specify "current" instead of a gamePk for a player\'s current game stats.'}

In [287]:
mlb.ENDPOINTS['meta']

{'url': 'https://statsapi.mlb.com/api/{ver}/{type}',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'type': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': [[]],
 'required_params': [[]],
 'note': 'The meta endpoint is used to retrieve values to be used within other API calls. Available types: awards, baseballStats, eventTypes, gameStatus, gameTypes, hitTrajectories, jobTypes, languages, leagueLeaderTypes, logicalEvents, metrics, pitchCodes, pitchTypes, platforms, positions, reviewReasons, rosterTypes, scheduleEventTypes, situationCodes, sky, standingsTypes, statGroups, statTypes, windDirection.'}

In [9]:
mlb.get('meta',{'type':'doubleHeader'})

ValueError: Request failed. Status Code: 404.

In [164]:
def get_gamePks(seasons):
    """
    Returns a list of gamePks for non-exhibition games given a list of seasons as integers.
    """
    import statsapi as mlb
    import csv
    import time
    import sys
    
    gamePks_path = "/Users/schlinkertc/code/mlb_predictions/gamePks"
    
    from os import walk
    import re
    
    # walk the gamePks directory to find the seasons that we've already added 
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    years = [re.findall('[^.csv]+',x) for x in f]
    already_added = [int(item) for sublist in years for item in sublist if item[0] in ['1','2']]
    
    seasons = list(set(seasons)-set(already_added))
    
    gamePks = {}
    for season in seasons:
        mlb.get('season',{'sportId':1,'seasonId':str(season)})
        try:
            games = mlb.schedule(start_date=f'02/01/{season}',
                                    end_date=f'11/30/{season}',
                                    sportId=1)
            pks = [x['game_id'] for x in games]
            print(pks[0])
            
            gamePks[season]=pks
            print(len(gamePks))
            with open(gamePks_path + f'/{season}.csv', 'w',newline='') as myfile:
                wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                wr.writerow(gamePks[season])
        except ValueError as err:
            print(f'{season} failed. Error: {err} Waiting 10 seconds before resuming')
            time.sleep(10)
            #seasons.append(season)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise
            
        
    return gamePks   

In [165]:
seasons = list(range(2009,2020))

In [167]:
pks = get_gamePks(seasons)

In [175]:
gamePks_path = "/Users/schlinkertc/code/mlb_predictions/gamePks"
f = []
for (dirpath, dirnames, filenames) in walk(gamePks_path):
    f.extend(filenames)
    break
pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
pk_paths

['/Users/schlinkertc/code/mlb_predictions/gamePks/2009.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2019.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2018.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2015.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2014.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2016.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2017.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2013.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2012.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2010.csv',
 '/Users/schlinkertc/code/mlb_predictions/gamePks/2011.csv']

In [189]:
seasons = [item for sublist in [re.findall('/gamePks/([^.csv]+)',x) for x in pk_paths] for item in sublist]

In [191]:
season = re.findall('/gamePks/([^.csv]+)',pk_paths[0])

In [192]:
season

['2009']

In [176]:

gamePks = {}
with open(pk_paths[0], 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

In [4]:
def read_gamePks():
    gamePks_path = "/Users/schlinkertc/code/mlb_predictions/gamePks"
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
    
    gamePks = {}
    for path in pk_paths:
        season = re.findall('/gamePks/([^.csv]+)',path)
        with open(path, 'r') as f:
            reader = csv.reader(f)
            seasonPks = list(reader)
        gamePks[season[0]] = [item for sublist in seasonPks for item in sublist]
    return gamePks

In [29]:
gamePk_dict = read_gamePks()

In [30]:
test_gamePk = gamePk_dict['2019'][1000]

In [86]:
game_test = mlb.get('game',{'gamePk':566651})

In [87]:
game_test.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [88]:
game_test['liveData']['plays']

{'allPlays': [{'result': {'type': 'atBat',
    'event': 'Groundout',
    'eventType': 'field_out',
    'description': 'Ehire Adrianza grounds out, third baseman Matt Duffy to first baseman Yandy Diaz.',
    'rbi': 0,
    'awayScore': 0,
    'homeScore': 0},
   'about': {'atBatIndex': 0,
    'halfInning': 'top',
    'inning': 1,
    'startTime': '1970-01-01T00:00:00.000Z',
    'endTime': '2019-02-28T23:37:05.000Z',
    'isComplete': True,
    'isScoringPlay': False,
    'hasReview': False,
    'hasOut': True,
    'captivatingIndex': 0},
   'count': {'balls': 0, 'strikes': 0, 'outs': 1},
   'matchup': {'batter': {'id': 501303,
     'fullName': 'Ehire Adrianza',
     'link': '/api/v1/people/501303'},
    'batSide': {'code': 'R', 'description': 'Right'},
    'pitcher': {'id': 605483,
     'fullName': 'Blake Snell',
     'link': '/api/v1/people/605483'},
    'pitchHand': {'code': 'L', 'description': 'Left'},
    'batterHotColdZones': [],
    'pitcherHotColdZones': [],
    'splits': {'batter

In [33]:
[x for x in game_test['gameData'].keys()]

['game',
 'datetime',
 'status',
 'teams',
 'players',
 'venue',
 'weather',
 'review',
 'flags',
 'alerts',
 'probablePitchers',
 'officialScorer',
 'primaryDatacaster']

In [34]:
game_test['gameData']['game']

{'pk': 566781,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/05/04/tormlb-texmlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566781-2019-05-04',
 'season': '2019',
 'seasonDisplay': '2019'}

In [35]:
game_test['gameData']['datetime']

{'dateTime': '2019-05-05T00:05:00Z',
 'originalDate': '2019-05-04',
 'dayNight': 'night',
 'time': '7:05',
 'ampm': 'PM'}

In [69]:
test_date = game_test['gameData']['datetime']['dateTime']

In [70]:
datetime.strptime(test_date,datetime.strptime(test_date,'%Y-%m-%dT%H:%M:%SZ'))

datetime.datetime(2019, 5, 5, 0, 5)

In [62]:
test_time = game_test['gameData']['datetime']['time']+game_test['gameData']['datetime']['ampm']
test_time

'7:05PM'

In [66]:
import time

In [74]:
datetime.strptime(test_time,"%H:%M%p").time()

datetime.time(7, 5)

In [36]:
game_test['gameData']['status']

{'abstractGameState': 'Final',
 'codedGameState': 'F',
 'detailedState': 'Final',
 'statusCode': 'F',
 'abstractGameCode': 'F'}

In [90]:
game_test['gameData']['teams']['home']['id']

139

In [91]:
game_test['gameData']['teams']['away']['id']

142

In [92]:
gameData = game_test['gameData']

In [93]:
gameData['venue']

{'id': 2534,
 'name': 'Charlotte Sports Park',
 'link': '/api/v1/venues/2534',
 'location': {'city': 'Port Charlotte',
  'state': 'Florida',
  'stateAbbrev': 'FL',
  'defaultCoordinates': {'latitude': 26.99888, 'longitude': -82.18212}},
 'timeZone': {'id': 'America/New_York', 'offset': -5, 'tz': 'EST'},
 'fieldInfo': {'capacity': 6000,
  'turfType': 'Grass',
  'roofType': 'Open',
  'leftLine': 343,
  'center': 414,
  'rightLine': 343}}

In [94]:
gameData['probablePitchers']

{'away': {'id': 606167,
  'fullName': 'Mejia, Adalberto',
  'link': '/api/v1/people/606167'},
 'home': {'id': 605483,
  'fullName': 'Snell, Blake',
  'link': '/api/v1/people/605483'}}

In [96]:
game_test.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

## plays

In [254]:
play_test = game_test['liveData']['plays']['allPlays'][25]

In [269]:
play_test.keys()

dict_keys(['result', 'about', 'count', 'matchup', 'pitchIndex', 'actionIndex', 'runnerIndex', 'runners', 'playEvents', 'atBatIndex', 'playEndTime'])

In [255]:
play_test['result']

{'type': 'atBat',
 'event': 'Grounded Into DP',
 'eventType': 'grounded_into_double_play',
 'description': 'Jake Cave grounds into a double play, pitcher Colin Poche to second baseman Brandon Lowe to first baseman Yandy Diaz.   C.J. Cron out at 2nd.    Jake Cave out at 1st.',
 'rbi': 0,
 'awayScore': 0,
 'homeScore': 0}

In [256]:
play_test['about']

{'atBatIndex': 25,
 'halfInning': 'top',
 'inning': 4,
 'startTime': '2019-03-01T00:43:41.000Z',
 'endTime': '2019-03-01T00:45:54.000Z',
 'isComplete': True,
 'isScoringPlay': False,
 'hasReview': False,
 'hasOut': True,
 'captivatingIndex': 24}

In [257]:
play_test['matchup']

{'batter': {'id': 595909,
  'fullName': 'Jake Cave',
  'link': '/api/v1/people/595909'},
 'batSide': {'code': 'L', 'description': 'Left'},
 'pitcher': {'id': 621363,
  'fullName': 'Colin Poche',
  'link': '/api/v1/people/621363'},
 'pitchHand': {'code': 'L', 'description': 'Left'},
 'batterHotColdZones': [],
 'pitcherHotColdZones': [],
 'splits': {'batter': 'vs_LHP', 'pitcher': 'vs_LHB', 'menOnBase': 'Empty'}}

In [258]:
play_test['pitchIndex']

[1]

In [259]:
play_test['runnerIndex']

[0, 1]

In [260]:
play_test['actionIndex']

[0]

In [261]:
play_test['count']

{'balls': 0, 'strikes': 0, 'outs': 3}

In [267]:
game_test['gameData']['game']

{'pk': 566651,
 'type': 'S',
 'doubleHeader': 'N',
 'id': '2019/02/28/minmlb-tbamlb-1',
 'gamedayType': 'Y',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566651-2019-02-28',
 'season': '2019',
 'seasonDisplay': '2019'}

In [ ]:
plays_test = game_test['liveData']['plays']['allPlays'][x]

In [ ]:
def get_pitches(api_call):
    """
    Takes in the result of of mlbstats-api call on the game endpoint. 
    Returns list of dictionaries for each play
    """
    gameData = api_call['gameData']
    game = gameData['game']
    
    liveData = api_call['liveData']
    
    

## pitches

In [262]:
[play_test['playEvents'][x] for x in play_test['pitchIndex']]

[{'details': {'call': {'code': 'X', 'description': 'Hit Into Play - Out(s)'},
   'description': 'In play, out(s)',
   'code': 'X',
   'ballColor': 'rgba(26, 86, 190, 1.0)',
   'isInPlay': True,
   'isStrike': False,
   'isBall': False,
   'hasReview': False},
  'count': {'balls': 0, 'strikes': 0},
  'pitchData': {'strikeZoneTop': 3.359,
   'strikeZoneBottom': 1.7,
   'coordinates': {'x': 107.45, 'y': 172.0},
   'breaks': {}},
  'hitData': {'trajectory': 'ground_ball',
   'hardness': 'medium',
   'location': '1',
   'coordinates': {'coordX': 137.41, 'coordY': 156.27}},
  'index': 1,
  'playId': '05666516-0266-0013-000c-f08cd117d70a',
  'pitchNumber': 1,
  'startTime': '2019-03-01T00:45:36.000Z',
  'endTime': '2019-03-01T00:45:54.000Z',
  'isPitch': True,
  'type': 'pitch'}]

## Actions 

In [263]:
[play_test['playEvents'][x] for x in play_test['actionIndex']]

[{'details': {'description': 'Mound Visit.',
   'event': 'Game Advisory',
   'awayScore': 0,
   'homeScore': 0,
   'isScoringPlay': False,
   'hasReview': False},
  'count': {'balls': 0, 'strikes': 0, 'outs': 1},
  'index': 0,
  'startTime': '2019-03-01T00:43:46.000Z',
  'endTime': '2019-03-01T00:45:36.000Z',
  'isPitch': False,
  'type': 'action',
  'player': {'id': 595909, 'link': '/api/v1/people/595909'}}]

## Runners

In [273]:
len(play_test['runners'])

2

In [282]:
play_test['runners'][0]['movement']['start']

'1B'

In [283]:
[play_test['runners'][x]['movement']['start'] for x in play_test['runnerIndex']]

['1B', None]

## Teams

In [361]:
mlb.ENDPOINTS['team']

{'url': 'https://statsapi.mlb.com/api/{ver}/teams/{teamId}',
 'path_params': {'ver': {'type': 'str',
   'default': 'v1',
   'leading_slash': False,
   'trailing_slash': False,
   'required': True},
  'teamId': {'type': 'str',
   'default': None,
   'leading_slash': False,
   'trailing_slash': False,
   'required': True}},
 'query_params': ['season', 'sportId', 'hydrate', 'fields'],
 'required_params': [[]]}

In [23]:
team_test = mlb.get('team',{'teamId':120})

In [29]:
team_test['teams'][0]['division']

{'id': 204, 'name': 'National League East', 'link': '/api/v1/divisions/204'}

## Investigate a more effecient way of getting player's performance up to a game

In [11]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()

In [12]:
# this is what we have now
query = session.query(Game,Play).\
                filter(Game.id==Play.game_id).\
                filter(and_(Game.type=='R',Game.dateTime<date)).\
                filter(Play.pitcher_id==self.id).\
                order_by(Game.dateTime.desc()).\
                all()[:limit]

In [76]:
def player_totals(self,game_record,session,limit=1000):    
    import numpy as np
    date=datetime.strftime(game_record.dateTime,'%Y%m%d')
    
    probable_starters = [game_record.home_probablePitcher,game_record.away_probablePitcher]
    stat_line = {}
    
    if self.primaryPosition_type in ['Hitter','Outfielder','Infielder','Catcher']:
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.batter_id={self.id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
    
    if self.primaryPosition_type=='Pitcher':
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.pitcher_id={self.id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
        
    plays=([
        {'game_id':q[0],'play_id':q[1],
         'event':q[2],'halfInning':q[3],
         'inning':q[4],'atBatIndex':q[5],
         'hasOut':q[6],'RBI':q[7],
         'count_outs':q[8],
         'num_pitches':q[9]} 
        for q in query])
    games={x[0] for x in query}

    
    singles = [x for x in plays if x['event']=='Single']
    doubles = [x for x in plays if x['event']=='Double']
    triples = [x for x in plays if x['event']=='Triple']
    home_runs = [x for x in plays if x['event']=='Home Run']
    walks = [x for x in plays if x['event']=='Walk']
    strikeouts = [x for x in plays if x['event'] in ['Strikeout','Strikeout Double Play']]
    HBP = [x for x in plays if x['event']=='Hit By Pitch']
    GDP = [x for x in plays if x['event']=='Grounded Into DP']
    IBB = [x for x in plays if x['event']=='Intent Walk']
    sac = [x for x in plays if x['event'] in ['Sac Bunt','Sac Fly']]
    interference = [x for x in plays if x['event']=='Catcher Interference']
    LO = [x for x in plays if x['event']=='Lineout']
    PO = [x for x in plays if x['event']=='Pop Out']
    FO = [x for x in plays if x['event']=='Flyout']
    GO = [x for x in plays if x['event']=='Groundout']
    
    ## pitcher stats 
    #starts
    #for hitters, this will mean they're the leadoff batter
    GS = {(x['inning'],x['game_id']) for x in plays if x['inning']==1}
    #games finished isn't adding up correctly yet. I think it's because walk-offs aren't counted
#     GF = {(x['inning'],x['game_id']) for x in plays 
#           if x['inning']==max([p.inning for p in x.game.plays]) 
#           and x.count_outs==3}        
    plays_with_outs = [x for x in plays if x['hasOut']==True]
    double_plays = [x for x in plays if x['event']=='Grounded Into DP' or 'Double Play' in x['event']]
    triple_plays = [x for x in plays if x['event']=='Triple Play']
    outs = len(plays_with_outs)+len(double_plays)+(len(triple_plays)*2)
    
    stat_line['Position_type']=self.primaryPosition_type
    stat_line['games']=len(games)
    stat_line['PA'] = len(plays)
    stat_line['AB'] = len(plays)-len(walks)-len(HBP)-len(IBB)-len(sac)-len(interference)
    stat_line['hits']=len(singles)+len(doubles)+len(triples)+len(home_runs)
    stat_line['singles']=len(singles)
    stat_line['doubles']=len(doubles)
    stat_line['triples']=len(triples)
    stat_line['home_runs']=len(home_runs)
    stat_line['walks']=len(walks)
    stat_line['strikeouts']=len(strikeouts)
    stat_line['HBP']=len(HBP)
    stat_line['RBIs']=sum([x['RBI'] for x in plays])
    stat_line['GDP']=len(GDP)
    stat_line['IBB']=len(IBB)
    stat_line['sac']=len(sac)
    stat_line['LO']=len(LO)
    stat_line['PO']=len(PO)
    stat_line['FO']=len(FO)
    stat_line['GO']=len(GO)
    stat_line['pitches_perPA']=np.mean([x['num_pitches'] for x in plays])
    
    stat_line['GS']=len(GS)
    #stat_line['GF']=len(GF)
        
    if 'Pitcher' in stat_line['Position_type']:
        stat_line['IP']=outs/3
    # is this pitcher generally a starter?    
    # pitchers who start the game less than 12th of their appearances are labeled relief
        if stat_line['GS']<stat_line['games']/12:
            stat_line['Position_type']='Pitcher_relief'
        else:
            stat_line['Position_type']='Pitcher_starter'
    
    # is this pitcher TODAY's starter?
    if self.id in probable_starters:
        stat_line['Position_type']='Probable_startingPitcher'            
    
    if 'Pitcher' not in stat_line['Position_type']:
        stat_line['IP']=0
    #saves/holds
    #stats for relief pitchers
    #stat_line['save_situations']=len([x for x in inning_score])       
    return stat_line

In [23]:
Base.metadata.clear()

In [24]:
class Cumulative_Stats(Base):
    __tablename__ = 'cumulative_stats'
    __table_args__ = {'extend_existing': True}
    
    player_id = Column(Integer,ForeignKey('people.id'),primary_key=True)
    game_id = Column(String(150),ForeignKey('plays.game_id'),primary_key=True)
    
    position_type = Column(String(20))
    games = Column(Integer)
    PA = Column(Integer)
    AB = Column(Integer)
    hits = Column(Integer)
    singles = Column(Integer)
    doubles = Column(Integer)
    triples = Column(Integer)
    home_runs = Column(Integer)
    walks = Column(Integer)
    strikeouts = Column(Integer)
    HBP = Column(Integer)
    RBIs = Column(Integer)
    GDP = Column(Integer)
    IBB = Column(Integer)
    sac = Column(Integer)
    LO = Column(Integer)
    PO = Column(Integer)
    FO = Column(Integer)
    GO = Column(Integer)
    pitches_perAB = Column(Integer)
    GS = Column(Integer)
    IP = Column(Integer)
    
#     play = relationship('Play',back_populates='people')
#     player = relationship('Person',back_populates='plays',lazy='dynamic')
    
    def __repr__(self): 
        return "<Game(pk='%s',id='%s')>" % (
                        self.player_id, self.game_id)
    
        
# Person.plays = relationship("Cumulative_Stats",order_by=Play,back_populates='player',lazy='dynamic')
# Play.people = relationship("Cumulative_Stats",back_populates='play')

In [17]:
Base.metadata.clear()

In [3]:
batters= session.execute("""select
                                plays.batter_id,
                                people.primaryPosition_type,
                                plays.game_id,
                                plays.startTime
                            from 
                                MLB_Stats.plays
                            inner join
                                MLB_Stats.people
                                on 
                                plays.batter_id=people.id
                            order by 
                                plays.startTime desc; """).fetchall()

In [2]:
def create_addCumulative_Stats(session,row,limit=1000):    
    import numpy as np
    from datetime import datetime
    player_id = row[0]
    position = row[1]
    game_id = row[2]
    date = datetime.strftime(row[-1],"%Y%m%d")

    
    #probable_starters = [game_record.home_probablePitcher,game_record.away_probablePitcher]
    stat_line = {'game_id':game_id,'player_id':player_id}
    
    if position in ['Hitter','Outfielder','Infielder','Catcher']:
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.batter_id={player_id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
    
    if position=='Pitcher':
        query = session.execute(f"""select
            g.id,
            p.id,
            p.`event`,
            p.halfInning,
            p.inning,
            p.atBatIndex,
            p.hasOut,
            p.RBI,
            p.count_outs,
            p.num_pitches
        from 
            MLB_Stats.games g
        right join
            MLB_Stats.plays p
            on 
            g.id=p.game_id
        where
            p.pitcher_id={player_id}
            and 
            date(g.`dateTime`)<date({date})
            and
            g.type='R'
        order by 
            p.startTime desc;""").fetchall()
        
    plays=([
        {'game_id':q[0],'play_id':q[1],
         'event':q[2],'halfInning':q[3],
         'inning':q[4],'atBatIndex':q[5],
         'hasOut':q[6],'RBI':q[7],
         'count_outs':q[8],
         'num_pitches':q[9]} 
        for q in query])
    games={x[0] for x in query}

    
    singles = [x for x in plays if x['event']=='Single']
    doubles = [x for x in plays if x['event']=='Double']
    triples = [x for x in plays if x['event']=='Triple']
    home_runs = [x for x in plays if x['event']=='Home Run']
    walks = [x for x in plays if x['event']=='Walk']
    strikeouts = [x for x in plays if x['event'] in ['Strikeout','Strikeout Double Play']]
    HBP = [x for x in plays if x['event']=='Hit By Pitch']
    GDP = [x for x in plays if x['event']=='Grounded Into DP']
    IBB = [x for x in plays if x['event']=='Intent Walk']
    sac = [x for x in plays if x['event'] in ['Sac Bunt','Sac Fly']]
    interference = [x for x in plays if x['event']=='Catcher Interference']
    LO = [x for x in plays if x['event']=='Lineout']
    PO = [x for x in plays if x['event']=='Pop Out']
    FO = [x for x in plays if x['event']=='Flyout']
    GO = [x for x in plays if x['event']=='Groundout']
    
    ## pitcher stats 
    #starts
    #for hitters, this will mean they're the leadoff batter
    GS = {(x['inning'],x['game_id']) for x in plays if x['inning']==1}
    #games finished isn't adding up correctly yet. I think it's because walk-offs aren't counted
#     GF = {(x['inning'],x['game_id']) for x in plays 
#           if x['inning']==max([p.inning for p in x.game.plays]) 
#           and x.count_outs==3}        
    plays_with_outs = [x for x in plays if x['hasOut']==True]
    double_plays = [x for x in plays if x['event']=='Grounded Into DP' or 'Double Play' in x['event']]
    triple_plays = [x for x in plays if x['event']=='Triple Play']
    outs = len(plays_with_outs)+len(double_plays)+(len(triple_plays)*2)
    
    stat_line['Position_type']=position
    stat_line['games']=len(games)
    stat_line['PA'] = len(plays)
    stat_line['AB'] = len(plays)-len(walks)-len(HBP)-len(IBB)-len(sac)-len(interference)
    stat_line['hits']=len(singles)+len(doubles)+len(triples)+len(home_runs)
    stat_line['singles']=len(singles)
    stat_line['doubles']=len(doubles)
    stat_line['triples']=len(triples)
    stat_line['home_runs']=len(home_runs)
    stat_line['walks']=len(walks)
    stat_line['strikeouts']=len(strikeouts)
    stat_line['HBP']=len(HBP)
    stat_line['RBIs']=sum([x['RBI'] for x in plays])
    stat_line['GDP']=len(GDP)
    stat_line['IBB']=len(IBB)
    stat_line['sac']=len(sac)
    stat_line['LO']=len(LO)
    stat_line['PO']=len(PO)
    stat_line['FO']=len(FO)
    stat_line['GO']=len(GO)
    stat_line['pitches_perPA']=np.mean([x['num_pitches'] for x in plays])
    
    stat_line['GS']=len(GS)
    #stat_line['GF']=len(GF)
        
    #if 'Pitcher' in stat_line['Position_type']:
        #stat_line['IP']=outs/3
    # is this pitcher generally a starter?    
    # pitchers who start the game less than 12th of their appearances are labeled relief
    if stat_line['GS']<stat_line['games']/12:
        stat_line['Position_type']='Pitcher_relief'
    else:
        stat_line['Position_type']='Pitcher_starter'
    
#     is this pitcher TODAY's starter?
#     if player_id.id in probable_starters:
#         stat_line['Position_type']='Probable_startingPitcher'            
    
    if 'Pitcher' not in stat_line['Position_type']:
        stat_line['IP']=0
    
    return stat_line

In [32]:
session.rollback()

In [4]:
stat_lines=[]
for chunk in sql_alch_schema.chunk(50,batters):
    count=1
    print(f'chunk{count} out of {batters}')
    stat_lines.append([create_addCumulative_Stats(session,x) for x in chunk])
    count+=1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [8]:
stat_line

{'game_id': '2019/10/30/wasmlb-houmlb-1',
 'player_id': 488726,
 'Position_type': 'Pitcher_starter',
 'games': 291,
 'PA': 1270,
 'AB': 1147,
 'hits': 355,
 'singles': 236,
 'doubles': 76,
 'triples': 4,
 'home_runs': 39,
 'walks': 96,
 'strikeouts': 126,
 'HBP': 12,
 'RBIs': 166,
 'GDP': 36,
 'IBB': 3,
 'sac': 11,
 'LO': 111,
 'PO': 47,
 'FO': 135,
 'GO': 295,
 'pitches_perPA': 3.767716535433071,
 'GS': 260}

In [9]:
Cumulative_Stats(    
    player_id = stat_line['player_id']
    game_id = stat_line['game_id']
    
    position_type = stat_line['Position_type']
    games = stat_line['games']
    PA = stat_line['PA']
    AB = stat_line['AB']
    hits = stat_line['hits']
    singles = stat_line['singles']
    doubles = stat_line['doubles']
    triples = stat_line['triples']
    home_runs = stat_line['']
    walks = Column(Integer)
    strikeouts = Column(Integer)
    HBP = Column(Integer)
    RBIs = Column(Integer)
    GDP = Column(Integer)
    IBB = Column(Integer)
    sac = Column(Integer)
    LO = Column(Integer)
    PO = Column(Integer)
    FO = Column(Integer)
    GO = Column(Integer)
    pitches_perAB = Column(Integer)
    GS = Column(Integer)
    IP = Column(Integer))

InvalidRequestError: Multiple classes found for path "Cumulative_Stats" in the registry of this declarative base. Please use a fully module-qualified path.